In [1]:
import os
import sqlalchemy
import pandas as pd
import networkx as nx
import holoviews as hv
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from bokeh.plotting import figure

In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql

In [3]:
database = "data/album.db"
con = sql.connect(database)

In [20]:
query = '''select user_a, user_b,
       sum(qtde_figurinhas) as qtde_figurinhas,
       sum(case when qtde_figurinhas = 0 then 0 else 1 end) as qtde_trocas,
       count(1) as qtde_tentativas

from tb_troca

where qtde_figurinhas > 0

group by  user_a, user_b
order by user_a, user_b'''

In [28]:
query2 = '''with tb_network as (
    select user_a,
        user_b,
        sum(qtde_figurinhas) as qtde_figurinhas,
        sum(case when qtde_figurinhas = 0 then 0 else 1 end) as qtde_trocas,
        count(1) as qtde_tentativas

    from tb_troca

    group by  user_a, user_b

    order by user_a, user_b )

select user,
       sum(qtde_relacionamentos) as qtde_relacionamentos,
       sum(qtde_figurinhas) as qtde_figurinhas

from (
    select user_a as user,
    qtde_figurinhas,
    (1) as qtde_relacionamentos
    from tb_network
    where qtde_figurinhas > 0

    UNION ALL

    select user_b as user,
    qtde_figurinhas,
    (1) as qtde_relacionamentos
    from tb_network
    where qtde_figurinhas > 0
)

group by user

order by qtde_relacionamentos desc'''

In [34]:
df = pd.read_sql_query(query, con)

In [35]:
df.head()

,user_a,user_b,qtde_figurinhas,qtde_trocas,qtde_tentativas
0,arnaldobjunior,costadanielc,1,1,1
1,arnaldobjunior,gabrielreiss,1,1,1
2,artesanah,costadanielc,5,1,1
3,artesanah,danilovboas,13,1,1
4,artesanah,ddauriol,2,1,1


In [36]:
df.shape

(85, 5)

In [37]:
grafico = nx.from_pandas_edgelist(df, source='user_a',target='user_b' )

In [38]:
st.bokeh_chart(hv.render(grafico, backend='bokeh'))

AttributeError: 'Graph' object has no attribute 'traverse'

In [ ]:
con.close